In [ ]:
import sys
# !{sys.executable} -m pip install 'pandas'
# !{sys.executable} -m pip install 'certifi' --upgrade


In [1]:
import requests
import certifi
import pandas as pd

## The purpose of this notebook is to showcase (And troubleshoot now) a API connection to the swagger - rest api that MGL uses.

### MGL Swagger page
https://malegislature.gov/api/swagger/index.html?url=/api/swagger/v1/swagger.json

### Requests docs:
https://docs.python-requests.org/en/latest/user/advanced/#ssl-cert-verification
https://docs.python-requests.org/en/latest/user/advanced/#ssl-cert-verification

### Troubleshooting docs:
https://stackoverflow.com/questions/54533595/curl-queries-work-but-python-request-fails
https://stackoverflow.com/questions/46604114/python-requests-ssl-error-certificate-verify-failed
https://stackoverflow.com/questions/27068163/python-requests-not-handling-missing-intermediate-certificate-only-from-one-mach/66111417#66111417

### Other docs:
https://www.geeksforgeeks.org/ssl-certificate-verification-python-requests/
https://levelup.gitconnected.com/solve-the-dreadful-certificate-issues-in-python-requests-module-2020d922c72f
https://www.reddit.com/r/learnpython/comments/q3tvxa/error_when_using_proxy_with_python_requests/
https://community.esri.com/t5/python-questions/solving-ssl-errors-in-python-requests/td-p/1124005

In [2]:
!pip list

Package            Version
------------------ ---------
certifi            2023.7.22
charset-normalizer 3.2.0
distlib            0.3.7
filelock           3.12.4
idna               3.4
pip                23.2.1
pipenv             2023.9.8
platformdirs       3.10.0
requests           2.31.0
setuptools         68.2.2
urllib3            2.0.4
virtualenv         20.24.5


In [3]:
import pandas as pd
import subprocess
import json

def fetch_data_with_curl(url):

    headers = ["accept: application/json"]
    
    try:
        result = subprocess.run(['curl', '-s', '-X', 'GET', url, '-H'] + headers, 
                                capture_output=True, check=True, text=True)

        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error fetching data from {url}: {e}") from e

url = "https://malegislature.gov/api/Chapters"
data = fetch_data_with_curl(url)

df = pd.DataFrame(data)

print(df)


     Code                                     Details
0       1     http://malegislature.gov/api/Chapters/1
1       2     http://malegislature.gov/api/Chapters/2
2       3     http://malegislature.gov/api/Chapters/3
3       4     http://malegislature.gov/api/Chapters/4
4       5     http://malegislature.gov/api/Chapters/5
..    ...                                         ...
695   23N   http://malegislature.gov/api/Chapters/23N
696  115B  http://malegislature.gov/api/Chapters/115B
697   40Y   http://malegislature.gov/api/Chapters/40Y
698  176X  http://malegislature.gov/api/Chapters/176X
699  131B  http://malegislature.gov/api/Chapters/131B

[700 rows x 2 columns]


In [4]:
df['Code']

0         1
1         2
2         3
3         4
4         5
       ... 
695     23N
696    115B
697     40Y
698    176X
699    131B
Name: Code, Length: 700, dtype: object

In [5]:
# Pull and retrieve all chapters in json
chapters_request_url = "https://malegislature.gov/api/Chapters"

chapter_response = requests.get(chapters_request_url)

if chapter_response.status_code == 200:
    chapter_json = chapter_response.json()
else: chapter_response.raise_for_status()

print(chapter_json)

SSLError: HTTPSConnectionPool(host='malegislature.gov', port=443): Max retries exceeded with url: /api/Chapters (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [6]:
# path of cert
certifi.where()

'/opt/homebrew/opt/python-certifi/lib/python3.11/site-packages/certifi/cacert.pem'

In [7]:
# Attempt #2 with certifi
chapter_response = requests.get(chapters_request_url, verify=certifi.where())

if chapter_response.status_code == 200:
    chapter_json = chapter_response.json()
else: chapter_response.raise_for_status()

print(chapter_json)

SSLError: HTTPSConnectionPool(host='malegislature.gov', port=443): Max retries exceeded with url: /api/Chapters (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [8]:
# Attempt #3 with verify off
chapter_response = requests.get(chapters_request_url, verify=False)

if chapter_response.status_code == 200:
    chapter_json = chapter_response.json()
else: chapter_response.raise_for_status()

print(chapter_json)

[{'Code': '1', 'Details': 'http://malegislature.gov/api/Chapters/1'}, {'Code': '2', 'Details': 'http://malegislature.gov/api/Chapters/2'}, {'Code': '3', 'Details': 'http://malegislature.gov/api/Chapters/3'}, {'Code': '4', 'Details': 'http://malegislature.gov/api/Chapters/4'}, {'Code': '5', 'Details': 'http://malegislature.gov/api/Chapters/5'}, {'Code': '6', 'Details': 'http://malegislature.gov/api/Chapters/6'}, {'Code': '6A', 'Details': 'http://malegislature.gov/api/Chapters/6A'}, {'Code': '6B', 'Details': 'http://malegislature.gov/api/Chapters/6B'}, {'Code': '6C', 'Details': 'http://malegislature.gov/api/Chapters/6C'}, {'Code': '6D', 'Details': 'http://malegislature.gov/api/Chapters/6D'}, {'Code': '6E', 'Details': 'http://malegislature.gov/api/Chapters/6E'}, {'Code': '7', 'Details': 'http://malegislature.gov/api/Chapters/7'}, {'Code': '7A', 'Details': 'http://malegislature.gov/api/Chapters/7A'}, {'Code': '7B', 'Details': 'http://malegislature.gov/api/Chapters/7B'}, {'Code': '7C', 'Det

/opt/homebrew/Cellar/jupyterlab/4.0.6/libexec/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [9]:
# Create dataframe from request
df = pd.DataFrame(chapter_json)

# make a list of chapters that are relevant and apply a filter
mgl_chapters = ["90", "263", "263A", "264", "265", "266", "267", "267A", "268", "268A", "268B", "269", "270", "271", "271A", "272", "273", "273A", "274"]
filtered_df = df[df['Code'].isin(mgl_chapters)]

filtered_df

,Code,Details
248,90,http://malegislature.gov/api/Chapters/90
662,263,http://malegislature.gov/api/Chapters/263
663,263A,http://malegislature.gov/api/Chapters/263A
664,264,http://malegislature.gov/api/Chapters/264
665,265,http://malegislature.gov/api/Chapters/265
666,266,http://malegislature.gov/api/Chapters/266
667,267,http://malegislature.gov/api/Chapters/267
668,267A,http://malegislature.gov/api/Chapters/267A
669,268,http://malegislature.gov/api/Chapters/268
670,268A,http://malegislature.gov/api/Chapters/268A


In [10]:
# Test of pulling all chapter 90 sections
c90_request_url = "https://malegislature.gov/api/Chapters/90/Sections"
c90_response = requests.get(c90_request_url, verify=False)

if c90_response.status_code == 200:
    c90_json = c90_response.json()
else: c90_response.raise_for_status()

df2 = pd.DataFrame(c90_json)
df2



/opt/homebrew/Cellar/jupyterlab/4.0.6/libexec/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Code,ChapterCode,Details
0,1,90,http://malegislature.gov/api/Chapters/90/Secti...
1,1A,90,http://malegislature.gov/api/Chapters/90/Secti...
2,1B,90,http://malegislature.gov/api/Chapters/90/Secti...
3,1C,90,http://malegislature.gov/api/Chapters/90/Secti...
4,1D,90,http://malegislature.gov/api/Chapters/90/Secti...
...,...,...,...
297,62,90,http://malegislature.gov/api/Chapters/90/Secti...
298,63,90,http://malegislature.gov/api/Chapters/90/Secti...
299,221/2,90,http://malegislature.gov/api/Chapters/90/Secti...
300,241/2,90,http://malegislature.gov/api/Chapters/90/Secti...


In [11]:
# list of c90 sections
df2['Code'].to_list()

['1',
 '1A',
 '1B',
 '1C',
 '1D',
 '1E',
 '1F',
 '1G',
 '1H',
 '1I',
 '2',
 '2A',
 '2B',
 '2C',
 '2D',
 '2E',
 '2F',
 '2G',
 '2H',
 '2I',
 '3',
 '3A',
 '3B',
 '3C',
 '3D',
 '3E',
 '3F',
 '3G',
 '4',
 '5',
 '5A',
 '5B',
 '6',
 '6A',
 '6B',
 '6C',
 '7',
 '7A',
 '7AA',
 '7B',
 '7BB',
 '7C',
 '7CC',
 '7D',
 '7D1/2',
 '7E',
 '7F',
 '7G',
 '7H',
 '7I',
 '7J',
 '7K',
 '7L',
 '7M',
 '7N',
 '7N1/2',
 '7N1/4',
 '7O',
 '7P',
 '7Q',
 '7R',
 '7R1/2',
 '7S',
 '7T',
 '7U',
 '7V',
 '7W',
 '7X',
 '7Y',
 '7Z',
 '8',
 '8A',
 '8A.5',
 '8A1/2',
 '8B',
 '8C',
 '8D',
 '8E',
 '8F',
 '8G',
 '8H',
 '8I',
 '8J',
 '8K',
 '8L',
 '8M',
 '9',
 '9A',
 '9B',
 '9C',
 '9D',
 '10',
 '10A',
 '11',
 '12',
 '12A',
 '13',
 '13A',
 '13B',
 '13C',
 '14',
 '14A',
 '14B',
 '15',
 '16',
 '16A',
 '16B',
 '17',
 '17A',
 '17B',
 '17C',
 '17D',
 '18',
 '18A',
 '18B',
 '19',
 '19A',
 '19B',
 '19C',
 '19D',
 '19E',
 '19F',
 '19G',
 '19H',
 '19I',
 '19J',
 '19K',
 '19L',
 '20',
 '20A',
 '20A1/2',
 '20B',
 '20C',
 '20D',
 '20E',
 '20F',


In [12]:
# Test of pulling c90s10

c90s10_request_url = "https://malegislature.gov/api/Chapters/90/Sections/10"
c90s10_response = requests.get(c90s10_request_url, verify=False)

if c90s10_response.status_code == 200:
    c90s10_json = c90s10_response.json()
else: c90s10_response.raise_for_status()

df3 = pd.DataFrame(c90s10_json)

/opt/homebrew/Cellar/jupyterlab/4.0.6/libexec/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'malegislature.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
df3

,Code,Name,IsRepealed,Text,Chapter,Part
Code,10,Operation of motor vehicle without license; me...,False,Section 10. No person under sixteen years of a...,90,I
Details,10,Operation of motor vehicle without license; me...,False,Section 10. No person under sixteen years of a...,http://malegislature.gov/api/Chapters/90,http://malegislature.gov/api/Parts/I


In [14]:
# pull raw string of pd's to get formatted into a list, dropping extra characters
raw_string = """
    Abington Police Department
    Acton Police Department
    Acushnet Police Department
    Adams Police Department[7]
    Agawam Police Department[8]
    Alford Police Department
    Andover Police Department
    Amesbury Police Department
    Amherst Police Department
    Arlington Police Department
    Ashburnham Police Department
    Ashby Police Department
    Ashfield Police Department
    Ashland Police Department
    Aquinnah Police Department
    Athol Police Department
    Attleboro Police Department[9]
    Auburn Police Department
    Avon Police Department
    Ayer Police Department
    Barnstable Police Department[10]
    Barre Police Department
    Becket Police Department
    Bedford Police Department[11]
    Belchertown Police Department
    Bellingham Police Department
    Belmont Police Department
    Berkley Police Department[citation needed]
    Berlin Police Department[citation needed]
    Bernardston Police Department
    Beverly Police Department
    Billerica Police Department
    Blackstone Police Department[citation needed]
    Blandford Police Department[12]
    Bolton Police Department
    Boston Police Department[13]
    Bourne Police Department[14]
    Boxborough Police Department
    Boxford Police Department
    Boylston Police Department
    Braintree Police Department[15]
    Brewster Police Department
    Bridgewater Police Department
    Brimfield Police Department
    Brockton Police Department[16]
    Brookfield Police Department
    Brookline Police Department
    Buckland Police Department
    Burlington Police Department
    Cambridge Police Department[17]
    Canton Police Department
    Carlisle Police Department
    Carver Police Department
    Charlemont Police Department
    Charlton Police Department
    Chatham Police Department
    Chelmsford Police Department
    Chelsea Police Department[18]
    Cheshire Police Department
    Chester Police Department
    Chesterfield Police Department
    Chicopee Police Department[19]
    Chilmark Police Department
    Clarksburg Police Department
    Clinton Police Department
    Cohasset Police Department
    Colrain Police Department
    Concord Police Department
    Conway Police Department
    Cummington Police Department
    Dalton Police Department
    Danvers Police Department
    Dartmouth Police Department
    Dedham Police Department
    Deerfield Police Department
    Dennis Police Department
    Dighton Police Department
    Douglas Police Department
    Dover Police Department
    Dracut Police Department
    Dudley Police Department
    Dunstable Police Department
    Duxbury Police Department
    East Bridgewater Police Department
    East Brookfield Police Department
    East Longmeadow Police Department
    Eastham Police Department
    Easthampton Police Department
    Easton Police Department
    Edgartown Police Department
    Egremont Police Department
    Erving Police Department
    Essex Police Department
    Everett Police Department[20]
    Fairhaven Police Department
    Fall River Police Department[21]
    Falmouth Police Department[22]
    Fitchburg Police Department[23]
    Foxborough Police Department
    Framingham Police Department
    Franklin Police Department[24]
    Freetown Police Department
    Gardner Police Department
    Georgetown Police Department
    Gill Police Department
    Gloucester Police Department
    Goshen Police Department
    Gosnold Police Department
    Grafton Police Department
    Granby Police Department
    Granville Police Department
    Great Barrington Police Department
    Greenfield Police Department
    Groton Police Department
    Hadley Police Department
    Halifax Police Department
    Hamilton Police Department
    Hampden Police Department
    Hancock Police Department
    Hanover Police Department
    Hanson Police Department
    Hardwick Police Department
    Harvard Police Department
    Harwich Police Department
    Hatfield Police Department
    Haverhill Police Department[25]
    Hawley Police Department
    Heath Police Department
    Hingham Police Department
    Hinsdale Police Department
    Holbrook Police Department
    Holden Police Department
    Holland Police Department
    Holliston Police Department
    Holyoke Police Department[26]
    Hopedale Police Department
    Hopkinton Police Department
    Hubbardston Police Department
    Hudson Police Department
    Hull Police Department[27]
    Huntington Police Department
    Ipswich Police Department
    Kingston Police Department
    Lancaster Police Department
    Lanesborough Police Department
    Lakeville Police Department
    Lawrence Police Department[28]
    Lee Police Department[29]
    Leicester Police Department
    Lenox Police Department
    Leominster Police Department
    Leverett Police Department
    Lexington Police Department
    Leyden Police Department
    Lincoln Police Department
    Littleton Police Department
    Longmeadow Police Department
    Lowell Police Department[30]
    Ludlow Police Department
    Lunnenburg Police Department
    Lynn Police Department[31]
    Lynnfield Police Department
    Malden Police Department[32]
    Manchester-by-the-Sea Police Department
    Mansfield Police Department
    Marion Police Department
    Marblehead Police Department
    Marlborough Police Department[33]
    Marshfield Police Department
    Mashpee Police Department[citation needed]
    Mattapoisett Police Department[citation needed]
    Maynard Police Department
    Medfield Police Department
    Medford Police Department[34]
    Medway Police Department
    Melrose Police Department[35]
    Mendon Police Department
    Merrimac Police Department[citation needed]
    Methuen Police Department[36]
    Middleborough Police Department
    Middlefield Police Department
    Middleton Police Department
    Milford Police Department
    Millbury Police Department[37]
    Millis Police Department
    Milleville Police Department
    Milton Police Department
    Monson Police Department
    Montague Police Department
    Monterey Police Department
    Nahant Police Department
    Nantucket Police Department
    Natick Police Department
    Needham Police Department
    New Bedford Police Department[38]
    New Braintree Police Department
    New Marlborough Police Department
    New Salem Police Department
    Newbury Police Department
    Newburyport Police Department
    Newton Police Department[39]
    Norfolk Police Department
    North Adams Police Department[40]
    North Andover Police Department
    North Attleborough Police Department
    North Brookfield Police Department
    North Reading Police Department
    Northampton Police Department[41]
    Northborough Police Department
    Northbridge Police Department
    Northfield Police Department
    Norton Police Department
    Norwell Police Department[42]
    Norwood Police Department
    Oak Bluffs Police Department
    Oakham Police Department
    Orange Police Department
    Orleans Police Department
    Otis Police Department
    Oxford Police Department
    Palmer Police Department
    Paxton Police Department
    Peabody Police Department[43]
    Pepperell Police Department
    Pelham Police Department
    Pembroke Police Department
    Peru Police Department
    Petersham Police Department
    Phillipston Police Department
    Pittsfield Police Department[44]
    Plainfield Police Department
    Plainville Police Department
    Plymouth Police Department
    Plympton Police Department
    Princeton Police Department
    Provincetown Police Department[45]
    Quincy Police Department[46]
    Raynham Police Department
    Randolph Police Department
    Reading Police Department
    Rehoboth Police Department
    Revere Police Department[47]
    Richmond Police Department
    Rochester Police Department
    Rockland Police Department
    Rockport Police Department
    Rowe Police Department
    Rowley Police Department
    Royalston Police Department
    Russell Police Department
    Rutland Police Department
    Salem Police Department[48]
    Salisbury Police Department
    Sandisfield Police Department
    Sandwich Police Department[49]
    Saugus Police Department
    Savoy Police Department
    Scituate Police Department
    Seekonk Police Department[50]
    Sharon Police Department
    Sheffield Police Department
    Shelburne Police Department
    Sherborne Police Department
    Shirley Police Department
    Shrewsbury Police Department
    Shutesbury Police Department
    Somerset Police Department
    Somerville Police Department[51]
    South Hadley Police Department
    Southborough Police Department
    Southbridge Police Department
    Southampton Police Department
    Southwick Police Department
    Spencer Police Department
    Springfield Police Department[52]
    Sterling Police Department
    Stockbridge Police Department
    Stoneham Police Department
    Stoughton Police Department
    Stow Police Department
    Sturbridge Police Department
    Sudbury Police Department
    Sutton Police Department
    Swansea Police Department
    Swampscott Police Department
    Taunton Police Department[53]
    Templeton Police Department
    Tewksbury Police Department[54]
    Tisbury Police Department
    Tolland Police Department
    Topsfield Police Department
    Townsend Police Department
    Truro Police Department[55]
    Tynsborough Police Department
    Tyringham Police Department
    Upton Police Department
    Uxbridge Police Department
    Wakefield Police Department
    Wales Police Department
    Walpole Police Department
    Waltham Police Department[56]
    Ware Police Department
    Wareham Police Department[57]
    Warren Police Department
    Warwick Police Department
    Washington Police Department
    Watertown Police Department[58]
    Wayland Police Department
    Webster Police Department
    Wellesley Police Department
    Wellfleet Police Department[59]
    Wendell Police Department
    Wenham Police Department
    West Boylston Police Department
    West Bridgewater Police Department
    West Brookfield Police Department
    West Newbury Police Department
    West Springfield Police Department[60]
    West Stockbridge Police Department
    West Tisbury Police Department
    Westborough Police Department
    Westhampton Police Department
    Westfield Police Department[61]
    Westford Police Department
    Westminster Police Department
    Westport Police Department
    Weston Police Department
    Westwood Police Department
    Weymouth Police Department[62]
    Whately Police Department
    Whitman Police Department
    Wilbraham Police Department
    Williamsburg Police Department
    Williamstown Police Department
    Wilmington Police Department
    Winchendon Police Department
    Winchester Police Department
    Windsor Police Department
    Winthrop Police Department
    Woburn Police Department[63]
    Worcester Police Department[64]
    Worthington Police Department
    Wrentham Police Department
    Yarmouth Police Department[65]"""

In [15]:
lines = [line.strip() for line in raw_string.split('\n') if line.strip()]
town_police_departments = [line for line in lines if line.endswith(" Police Department")]

In [16]:
town_police_departments


['Abington Police Department',
 'Acton Police Department',
 'Acushnet Police Department',
 'Alford Police Department',
 'Andover Police Department',
 'Amesbury Police Department',
 'Amherst Police Department',
 'Arlington Police Department',
 'Ashburnham Police Department',
 'Ashby Police Department',
 'Ashfield Police Department',
 'Ashland Police Department',
 'Aquinnah Police Department',
 'Athol Police Department',
 'Auburn Police Department',
 'Avon Police Department',
 'Ayer Police Department',
 'Barre Police Department',
 'Becket Police Department',
 'Belchertown Police Department',
 'Bellingham Police Department',
 'Belmont Police Department',
 'Bernardston Police Department',
 'Beverly Police Department',
 'Billerica Police Department',
 'Bolton Police Department',
 'Boxborough Police Department',
 'Boxford Police Department',
 'Boylston Police Department',
 'Brewster Police Department',
 'Bridgewater Police Department',
 'Brimfield Police Department',
 'Brookfield Police Depa